In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [4]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from huggingface_hub import create_repo, upload_file
from datasets import load_dataset, DatasetDict, concatenate_datasets

In [5]:
ds_1 = load_dataset("Zaynoid/med-dpo-10k")

README.md:   0%|          | 0.00/354 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10815 [00:00<?, ? examples/s]

In [6]:
ds_1

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'question'],
        num_rows: 10815
    })
})

In [7]:
ds_2 = load_dataset("Julian2002/Medical-ChatBot-DPO")

README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/72.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/164101 [00:00<?, ? examples/s]

In [8]:
ds_2

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 164101
    })
})

In [9]:
ds_2 = ds_2.rename_column("prompt", "question")

In [10]:
ds_2

DatasetDict({
    train: Dataset({
        features: ['question', 'chosen', 'rejected'],
        num_rows: 164101
    })
})

# Filter Medical ChatBot DPO dataset

In [15]:
keywords = ['sleep', 'fitness', 'stress', 'heart', 'health', 'health care', 'personal', 'medical', 'cardio', 'medicine', 'exercise', 'smoking', 'smoker', 'alcohol', 'alcoholic', 'bmi', 'blood pressure', 'steps', 'step', 'run', 'jog', ' rem ', 'circadian', 'homeostatic', 'injury', ' train', 'etiology', 'etiological', 'recommend', 'advice', 'advise', 'assistance', 'assist', 'workout', 'work out', 'lifestyle', 'z-score', 'z score', 'athelete', 'athlete', 'sport', 'respiratory', 'fatigue', 'pressure', 'recover', 'hydrate', 'faint', 'drowsy', 'drowsiness', 'gym', 'muscle', 'sore', 'wake', 'rest', 'relax', 'insomnia', 'physic', 'care', 'calorie', 'fat loss', 'weight', 'height', 'mobility', 'activity', 'active']

In [16]:
# Function to check if any keyword is present in a record
def contains_keyword(example):
    for keyword in keywords:
        if keyword in example['question'].lower() or keyword in example['chosen'].lower() or keyword in example['rejected'].lower():
            return True
    return False

In [18]:
# Apply the filter to each split
for split in ds_2.keys():
    ds_2[split] = ds_2[split].filter(contains_keyword)
print(ds_2)

Filter:   0%|          | 0/164101 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'chosen', 'rejected'],
        num_rows: 74546
    })
})


In [19]:
dataset = DatasetDict({"train": concatenate_datasets([ds_1["train"], ds_2["train"]])})

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'question'],
        num_rows: 85361
    })
})

# Upload dataset to HF

In [21]:
repo_id = "johnjehiel/ph-llm-DPO"

In [26]:
train_parquet_path = "ph-llm-DPO-dataset_train.parquet"
dataset["train"].to_parquet(train_parquet_path)

Creating parquet from Arrow format:   0%|          | 0/86 [00:00<?, ?ba/s]

94796545

In [27]:
create_repo(repo_id=repo_id, token=HF_TOKEN, repo_type="dataset", exist_ok=True)

RepoUrl('https://huggingface.co/datasets/johnjehiel/ph-llm-DPO', endpoint='https://huggingface.co', repo_type='dataset', repo_id='johnjehiel/ph-llm-DPO')

In [28]:
upload_file(
    path_or_fileobj=train_parquet_path,
    path_in_repo="ph-llm-DPO-dataset_train.parquet",
    repo_id=repo_id,
    token=HF_TOKEN,
    repo_type="dataset",
    commit_message="Upload train split for personal health DPO dataset"
)

ph-llm-DPO-dataset_train.parquet:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/johnjehiel/ph-llm-DPO/commit/cc3d03ff354124f47d0559cfd56fe039ac0aa969', commit_message='Upload train split for personal health DPO dataset', commit_description='', oid='cc3d03ff354124f47d0559cfd56fe039ac0aa969', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/johnjehiel/ph-llm-DPO', endpoint='https://huggingface.co', repo_type='dataset', repo_id='johnjehiel/ph-llm-DPO'), pr_revision=None, pr_num=None)

# Verify Uploaded Data

In [29]:
ds = load_dataset(repo_id)

ph-llm-DPO-dataset_train.parquet:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [30]:
ds

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'question'],
        num_rows: 85361
    })
})